In [2]:
%load_ext autoreload
%autoreload 2

import os,sys,inspect
# current_dir = '/home/jiayuand/seqOED_variational/examples/location'
# parent_dir = '/home/jiayuand/seqOED_variational'

# current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
# parent_dir = os.path.dirname(os.path.dirname(current_dir))
parent_dir = '../../../../'
sys.path.insert(0, parent_dir) 

import numpy as np
# from scipy.stats import norm, beta, dirichlet
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from   torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
if torch.cuda.is_available():  
    dev = "cuda" 
else:  
    dev = "cpu"  
device = torch.device(dev) 
torch.set_default_device(device)
dtype = torch.float32
torch.set_default_dtype(dtype)

from vsOED import VSOED, PGvsOED, GMM_NET, NFs, POST_APPROX
from vsOED.utils import *
from vsOED.models import *

import dowel
import joblib

print('==============================================================================')
print('Please go to /home/<USER>/.local/lib/python3.9/site-packages/torch/storage.py')
print('and change def _load_from_bytes(b):')
print('to')
print(f'    return torch.load(io.BytesIO(b), map_location="{dev}")')
print('==============================================================================')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Please go to /home/<USER>/.local/lib/python3.9/site-packages/torch/storage.py
and change def _load_from_bytes(b):
to
    return torch.load(io.BytesIO(b), map_location="cpu")


# Only interested in parameter of interest (poi)

In [3]:
data_folder = parent_dir + './results/source/unimodel/poi/'
subfolders = {'rl': './RL_BOED/',
              'gmm': data_folder + 'GMM',
              'nfs': data_folder + 'NFs'}
postfix = {'gmm':{'terminal': '_5e-4', 'incre': ''}}

In [4]:
# pick a mode , id and horizon, and load the data
post_net_type = 'gmm'
mode = 'terminal'
id = 0
h = 30
subfolder = subfolders[post_net_type] + f'/{mode}_id{id}_h{h}/'
eval_data = torch.load(subfolder + 'evaluation.pt', map_location=device)
params = eval_data['params']
ds_hist = eval_data['ds_hist']
ys_hist = eval_data['ys_hist']
rewards_hist = eval_data['rewards_hist']
vsoed = joblib.load(subfolder + f'/itr_10000.pkl')

In [6]:
n_traj = 8000
n_design = vsoed.n_design
design_bounds = vsoed.design_bounds

def new_actor(*args, **kws):
    return torch.rand(n_traj, n_design) * (design_bounds[:, 1] - design_bounds[:, 0]) + design_bounds[:, 0]

vsoed.actor = new_actor

In [7]:
ret = vsoed.asses(n_traj, n_contrastive_sample=int(1e6), return_all=True)

  2%|▏         | 41/2000 [00:29<23:48,  1.37it/s]


KeyboardInterrupt: 

In [ ]:
rwds = torch.cumsum(ret['rewards_hist'][:, :-1], dim=1)

In [ ]:
mean = rwds.mean(dim=0).cpu().numpy()
se = (rwds.std(dim=0) / np.sqrt(len(rwds))).cpu().numpy()

In [ ]:
np.save('random.npy', {'mean':mean, 'se':se})